In [ ]:
!pip install sentence_transformers
!pip install chromadb
!pip install peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 1.5 MB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 675.8 kB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... -

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report
from sentence_transformers import SentenceTransformer
import pandas as pd
from chromadb.utils import embedding_functions
from tqdm import tqdm
import chromadb
import torch
from transformers import Trainer
from transformers import BertTokenizer, BertForSequenceClassification
import numpy as np
import pandas as pd

In [5]:
device = 'CUDA' if torch.cuda.is_available else 'CPU'
device

'CUDA'

In [6]:
import torch
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

<h2>Загрузка моделей LLM, retriver, intent recognition</h2>

In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

llm_model = AutoModelForCausalLM.from_pretrained("openchat/openchat-3.5-0106", device_map="auto", torch_dtype="auto")
llm_tokenizer = AutoTokenizer.from_pretrained("openchat/openchat-3.5-0106", use_fast=False)
generation_config = GenerationConfig.from_pretrained('openchat/openchat-3.5-0106')
generation_config.top_k= 40
generation_config.top_p= 0.9
generation_config.temperature= 0.3
generation_config.do_sample= True

config.json:   0%|          | 0.00/651 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/491 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
retriver_model = SentenceTransformer("intfloat/multilingual-e5-small")
# retriver_model = SentenceTransformer("hivaze/ru-e5-large")

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/160k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [9]:
BASE_MODEL_NAME = "cointegrated/rubert-tiny2"
MODEL_NAME = "/kaggle/input/intent-model/intent-model"
intent_model = BertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=3)
intent_tokenizer = BertTokenizer.from_pretrained(BASE_MODEL_NAME)

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


tokenizer_config.json:   0%|          | 0.00/401 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.74M [00:00<?, ?B/s]

In [ ]:
# !chroma run --path /db_path

<h2>Инициализация векторной базы данных, загрузка и векторизация данных</h2>

In [10]:
client = chromadb.PersistentClient(path="/database/data")
print('База данных подключена') 
# invest_qa_emb = client.create_collection(name=db_name, metadata={'hnsw:space': 'cosine'})
collection = client.create_collection(name="Invest_collection", embedding_function=embedding_functions.SentenceTransformerEmbeddingFunction(model_name="intfloat/multilingual-e5-small"), metadata={'hnsw:space': 'cosine'})
print('Конфигурация коллекции создана')
df = pd.read_excel('/kaggle/input/data-with-pair-qa/data.xlsx')
collection.add(
    documents=df.Answer.to_list(),
    metadatas=[{'question': q} for q in df.Theme.to_list()],
    ids=[str(i) for i in range(len(df))]
)
print('Данные загружены')

База данных подключена
Конфигурация коллекции создана


Batches:   0%|          | 0/45 [00:00<?, ?it/s]

Данные загружены


Временно не нужно

In [ ]:
# import torch
# from peft import PeftModel, PeftConfig
# from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig


# MODEL_NAME = "IlyaGusev/saiga_mistral_7b"
# config = PeftConfig.from_pretrained(MODEL_NAME)
# model = AutoModelForCausalLM.from_pretrained(
#     config.base_model_name_or_path,
#     load_in_8bit=False,
#     torch_dtype=torch.float16,
#     device_map="auto"
# )
# model = PeftModel.from_pretrained(
#     model,
#     MODEL_NAME,
#     torch_dtype=torch.float16
# )
# tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)
# generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
# print(generation_config)



<h2>Создания класса Conversation для инициализации промпта, хранение контекста</h2>

In [84]:
# DEFAULT_MESSAGE_TEMPLATE = "<s>{role}\n{content}</s>"
# DEFAULT_RESPONSE_TEMPLATE = "<s>bot\n"
DEFAULT_MESSAGE_TEMPLATE = "<|{role}|>\n{content}</s>\n"
DEFAULT_RESPONSE_TEMPLATE = "<|assistant|>\n"
DEFAULT_SYSTEM_PROMPT = "Ты интеллектуальный помощник по инвестициям.Ты должен отвечать на вопросы пользователей на РУССКОМ ЯЗЫКЕ, основываясь на предоставленных ответах."
class Conversation:
    def __init__(
        self,
        message_template=DEFAULT_MESSAGE_TEMPLATE,
        system_prompt=DEFAULT_SYSTEM_PROMPT,
        response_template=DEFAULT_RESPONSE_TEMPLATE,
        default_conversation_len = 17
    ):
        self.message_template = message_template
        self.response_template = response_template
        self.messages = [{
            "role": "system",
            "content": system_prompt
        }]
        self.default_conversation_len = default_conversation_len
    def check_len(self):
        if self.__len__()==self.default_conversation_len:
                del self.messages[1]
#                 del self.messages[1]
    def add_user_message(self, message):
        self.check_len()
        self.messages.append({
            "role": "user",
            "content": message
        })
    def add_answer_message(self, message):
        self.check_len()
        self.messages.append({
            "role": "user",
            "content": message
        })

    def add_bot_message(self, message):
        self.check_len()
        self.messages.append({
            "role": "bot",
            "content": message
        })
    def get_prompt(self, tokenizer):
        final_text = ""
        for message in self.messages:
            message_text = self.message_template.format(**message)
            final_text += message_text
        final_text += DEFAULT_RESPONSE_TEMPLATE
        return final_text.strip()
    def __len__(self):
        return len(self.messages)
def generate(model, tokenizer, prompt, generation_config):
    data = tokenizer(prompt, return_tensors="pt", add_special_tokens=False).to("cuda")
    data = {k: v.to("cuda") for k, v in data.items()}
    output_ids = model.generate(
        **data,
        generation_config=generation_config
    )[0]
    output_ids = output_ids[len(data["input_ids"][0]):]
    output = tokenizer.decode(output_ids, skip_special_tokens=True)
    return output.strip()

<h2>Функция генерации ответа, которая состоит из</h2>
<h3>1) Доставание 5 ближайщих соседей к вопросу</h3>
<h3>2) Классификация вопроса на 3 класса (0-полезный запрос, 1- запрос-доспрашивание, 2-вопрос не относящийся к теме инвестиций)</h3>
<h3>3) Генерация ответа в зависимости от класса вопроса</h3>

In [92]:
def get_answer(query,conversation):
    DEFAULT_USER_MESSAGE= '''Тебе будут предостоставлены ответы на похожие вопросы из базы знаний в формате (ответ: коэфициент релевантности вопросу пользователя). Если среди предоставленных ответов есть информация которая отвечает на вопрос, то составь красивый и понятный ответ для пользователя, а если среди ответов нет информации, которая отвечает на вопрос, то переспроси пользователя(уточни что он имел ввиду), ни в коем случае не придумывай информацию!'''
    DEFAULT_USER_FURTHER_QUESTIONING = '''Нужно объяснить более понятным языком, основываясь на ответы из базы данных'''
    def get_answers_from_db(query):
        collection = client.get_collection("Invest_collection")
        input_em = retriver_model.encode(sentences=f"query: {query}", batch_size=1).tolist()
        results = collection.query(
            query_embeddings=[input_em],
            n_results=5
        )
        answers = dict(zip(results['documents'][0], results['distances'][0]))
        for key, value in answers.items():
            answers[key] = 1 - value
#         answers = [key for key in answers.keys()]
        return answers
    class QueryDataset(torch.utils.data.Dataset):
        def __init__(self, encodings, labels=None):
            self.encodings = encodings
            self.labels = labels

        def __getitem__(self, idx):
            item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
            if self.labels:
                item["labels"] = torch.tensor(self.labels[idx])
            return item

        def __len__(self):
            return len(self.encodings["input_ids"])
    tokenized_query = intent_tokenizer([query], padding=True, truncation=True, max_length=512)
    tokenized_query_dataset = QueryDataset(encodings=tokenized_query)
    test_trainer = Trainer(intent_model)
    raw_pred, _, _ = test_trainer.predict(tokenized_query_dataset)
    y_pred = torch.argmax(torch.softmax(torch.tensor(raw_pred), dim=-1), axis=1)
    print(y_pred)
    if y_pred==0:
        answers = get_answers_from_db(query)
        prompt_query = DEFAULT_USER_MESSAGE+query+str(answers)
        conversation.add_user_message(prompt_query)
        prompt = conversation.get_prompt(llm_tokenizer)
        output = generate(llm_model, llm_tokenizer, prompt, generation_config)
        torch.cuda.empty_cache()
        return output
    elif y_pred==1:
        prompt_query = DEFAULT_USER_FURTHER_QUESTIONING+query
        conversation.add_user_message(prompt_query)   
        prompt = conversation.get_prompt(llm_tokenizer)
        output = generate(llm_model, llm_tokenizer, prompt, generation_config)
        torch.cuda.empty_cache()
        return output
    else:
        return 'Пожалуйста повторите ваш вопрос'

In [ ]:
get_answer('Я не совсем понял',conversation)

<h2>Запуск телеграм бота и использование функции генерации ответа</h2>

In [ ]:
!pip install aiogram==2.25.2
!pip install nest-asyncio
# !pip install aiohttp==3.8.6

In [ ]:
import logging
from aiogram import Bot, Dispatcher, types
from aiogram.dispatcher.filters.state import State, StatesGroup
from aiogram.contrib.middlewares.logging import LoggingMiddleware
from aiogram.types import InlineKeyboardButton, InlineKeyboardMarkup
from aiogram.types import ReplyKeyboardMarkup, KeyboardButton
# from aiogram.utils import executor

# Инициализация бота и диспетчера
API_TOKEN = '7093361951:AAFDy4EkMv6pIB7MdlLsJKSKdLYcOVxcBjE'
bot = Bot(token=API_TOKEN)
dp = Dispatcher(bot)
logging.basicConfig(level=logging.INFO)
dp.middleware.setup(LoggingMiddleware())

user_dialogs = {}
user_context = {}

# Обработчик команды /start
@dp.message_handler(commands=['start'])
async def start(message: types.Message):
    user_id = message.from_user.id
    user_dialogs[user_id] = []  # Создаем пустой диалог для пользователя
    keyboard = ReplyKeyboardMarkup(resize_keyboard=True).add(KeyboardButton("ОБЪЯСНИ"))
    await message.answer("Привет! Я диалоговый бот. Нажми кнопку 'ОБЪЯСНИ', чтобы начать диалог.", reply_markup=keyboard)

# Обработчик кнопки "ОБЪЯСНИ"
@dp.message_handler(lambda message: message.text == 'ОБЪЯСНИ')
async def explain(message: types.Message, state: FSMContext):
    user_id = message.from_user.id
    user_dialogs[user_id].append(["юзер", message.text])  # Добавляем сообщение пользователя в диалог
    user_context[user_id] = Conversation()
    keyboard = ReplyKeyboardMarkup(resize_keyboard=True).add(KeyboardButton("ПОНЯЛ"))
    await message.answer("Теперь я могу объяснить. Напиши свой вопрос:", reply_markup=keyboard)

# Обработчик всех сообщений внутри диалога
@dp.message_handler(lambda message: message.text != 'ПОНЯЛ', state="*")
async def handle_dialog(message: types.Message, state: FSMContext):
    user_id = message.from_user.id
    conversation = user_context.get(user_id)
    print(conversation)
    await message.answer('Ваш вопрос принят, ожидайте')
    answer = get_answer(message.text,conversation)
    print('Ответ готов')
    user_dialogs[user_id].append(["юзер", message.text])  # Добавляем сообщение пользователя в диалог
    user_dialogs[user_id].append(['бот', answer])
    await message.answer(answer)
    await message.answer("Продолжай задавать вопросы или нажми кнопку 'ПОНЯЛ', чтобы завершить диалог.")

# Обработчик кнопки "ПОНЯЛ"
@dp.message_handler(lambda message: message.text == 'ПОНЯЛ')
async def understood(message: types.Message):
    user_id = message.from_user.id
    dialog = user_dialogs.get(user_id, [])
    # Сохраняем диалог в файл (вместо этого можно использовать базу данных)
    with open(f"user_{user_id}_dialog.txt", "w", encoding='utf-8') as file:
        for entry in dialog:
            file.write(f"{entry[0]}: {entry[1]}\n")
    user_dialogs[user_id] = []  # Обнуляем диалог
    conversation = Conversation()
    keyboard = ReplyKeyboardMarkup(resize_keyboard=True).add(KeyboardButton("ОБЪЯСНИ"))
    await message.answer("Диалог сохранен. Начнем заново!", reply_markup=keyboard)


import nest_asyncio
nest_asyncio.apply()
if __name__ == '__main__':
    from aiogram import executor
    executor.start_polling(dp, skip_updates=True)

In [66]:
!pip install telebot

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 kB 688.2 kB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.2/232.2 kB 2.5 MB/s eta 0:00:00a 0:00:01


In [94]:
import telebot
import logging
import random

API_TOKEN = '7093361951:AAFDy4EkMv6pIB7MdlLsJKSKdLYcOVxcBjE'
bot = telebot.TeleBot(API_TOKEN)

user_dialogs = {}
user_context = {}

# Обработчик команды /start
@bot.message_handler(commands=['start'])
def start(message):
    user_id = message.from_user.id
    user_dialogs[user_id] = []  # Создаем пустой диалог для пользователя
    keyboard = telebot.types.ReplyKeyboardMarkup(resize_keyboard=True).add(telebot.types.KeyboardButton("Новый чат"))
    user_id = message.from_user.id
    user_dialogs[user_id].append(["юзер", message.text])
    user_context[user_id] = Conversation()
    bot.send_message(message.chat.id, "Привет! Я ваш инвестиционный помощник. Задавай свой вопрос и я постараюсь объяснить мир инвестиций максимально простым языком.", reply_markup=keyboard)

# Обработчик всех сообщений внутри диалога
@bot.message_handler(func=lambda message: message.text != 'Новый чат', content_types=['text'])
def handle_dialog(message):
    user_id = message.from_user.id
    conversation = user_context.get(user_id)
    print(len(conversation))
    answer = get_answer(message.text,conversation)
    conversation.add_bot_message(answer)
    print('Ответ готов')
    user_dialogs[user_id].append(["юзер", message.text])
    user_dialogs[user_id].append(['бот', answer])
    bot.send_message(message.chat.id, answer)
#     bot.send_message(message.chat.id, "Продолжай задавать вопросы или нажми кнопку 'ПОНЯЛ', чтобы завершить диалог.")

# Обработчик кнопки "ПОНЯЛ"
@bot.message_handler(func=lambda message: message.text == 'Новый чат')
def understood(message):
    user_id = message.from_user.id
    dialog = user_dialogs.get(user_id, [])
    # Сохраняем диалог в файл (вместо этого можно использовать базу данных)
    with open(f"user_{user_id}_dialog.txt", "w", encoding='utf-8') as file:
        for entry in dialog:
            file.write(f"{entry[0]}: {entry[1]}\n")
    user_dialogs[user_id] = []  # Обнуляем диалог
    conversation = Conversation()
    print(len(conversation))
    keyboard = telebot.types.ReplyKeyboardMarkup(resize_keyboard=True).add(telebot.types.KeyboardButton("Новый чат"))
    bot_message = ['Начинаем с чистого листа', 'Разговаривал бы с вами вечно', 'Что нибудь еще?', 'Спрашивайте, мне нравится вам помогать!', "Ух, я готов вам помочь"]
    bot.send_message(message.chat.id, random.choice(bot_message), reply_markup=keyboard)

if __name__ == '__main__':
    bot.polling(none_stop=True)


1


tensor([0])


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Ответ готов
3


tensor([0])


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Ответ готов
5


tensor([1])
Ответ готов
7


tensor([1])
Ответ готов
1
1
1
1
